# *libcusmm*: explore the parameter space 

## Notes

* smem usage as computed in Python script isn't correct: 
	* run ``nvcc -O3 -arch=sm_60 -Xptxas -v -w -c tune_25x32x5_exe0_part0.cu`` to find out 
	* It seems to be correct for tiny, but not for the other algorithms 
	* I cannot seem to find a pattern or understand the non-constant offset
* number of registers used should be predictor as well (obtain by above) 
* often, the two best performing configs have ``tile_mA * tile_nA = tile_mB * tile_nB``. Perhaps ``tile_m * tile_n` should be a predictor? 

#### Bounds and constraints

* Threads (min, max) 
	* rare cases in which going above brings ~4% perf improvement

* Max parallel work 

* tile_m, tile_n (min, max) 
	* values above: within noise level and in cases where bigger T schadet nicht

#### Stack size 

* used in autotuning: 16'005
* used in DBCSR with smm_acc: 30'000 (./core/dbcsr_config.F, mm_stack_default_size)
* A. B get chopped up and a list of stacks gets filled. Whenever it is full -> flush to multiply 
	* (there is a shorter list of remainder at the end) 

#### Todo 

* find constraints on w, v



# Read data

In [1]:
import numpy as np
import pandas as pd
import explore_parameters_utils as explore_utils

# Helper variables and functions
algo_dict = {'tiny': 0, 'small': 1, 'medium': 2, 'largeDB1': 3, 'largeDB2': 4}  # category-encoder 
algo_dict_rev = {0: 'tiny', 1: 'small', 2: 'medium', 3: 'largeDB1', 4: 'largeDB2'}
npars = 3   # number of parameters in stack list
def format_pars(df):
    df.replace(to_replace=algo_dict, inplace=True)
    df.fillna(value=0, inplace=True)
    df = df.rename(columns={'threads': 'threads_per_blk', 'nregs': 'regs_per_thread'})
    return df 

# Read GPU properties
gpu = pd.read_excel('parameters_P100.xlsx', sheet_name='GPU', header=0)
gpu = gpu.to_dict(orient='list')
for k, v in gpu.items():
    gpu[k] = v[0]

# Read autotuning settings
autotuning = pd.read_excel('parameters_P100.xlsx', sheet_name='Autotuning', header=0)
autotuning = autotuning.to_dict()
for k, v in autotuning.items():
    autotuning[k] = v[0]

# Read autotuning data
pars_autotuned = pd.read_excel('parameters_P100.xlsx', sheet_name='parameters_P100', header=0)
pars_autotuned = format_pars(pars_autotuned);
##mnk_kernel = 'tune_4x4x4_test-stacksize_1605'
##mnk_kernel = 'tune_4x4x5_test-stacksize_1605'
##mnk_kernel = 'tune_4x4x8_test-stacksize_1605'
##mnk_kernel = 'tune_4x4x32_test-stacksize_1605'
##mnk_kernel = 'tune_26x4x13'
mnk_kernel = 'tune_13x28x32'
##mnk_kernel = 'tune_13x28x45'
pars_autotuning = explore_utils.read_kernel_autotuning(mnk_kernel)
pars_autotuning = format_pars(pars_autotuning)

Loading data from tune_13x28x32/data_dump


In [2]:
print(gpu)

{'Physical Limits for GPU Compute Capability:': 6, 'Threads per Warp': 32, 'Max Warps per Multiprocessor': 64, 'Max Thread Blocks per Multiprocessor': 32, 'Max Threads per Multiprocessor': 2048, 'Maximum Thread Block Size': 1024, 'Registers per Multiprocessor': 65536, 'Max Registers per Thread Block': 65536, 'Max Registers per Thread': 255, 'Shared Memory per Multiprocessor (bytes)': 65536, 'Max Shared Memory per Block': 49152, 'Register allocation unit size': 256, 'Register allocation granularity': 'warp', 'Shared Memory allocation unit size': 256, 'Warp allocation granularity': 2, 'Multiprocessors': 56}


In [3]:
print(autotuning)

{'n_a': 10000, 'n_b': 10000, 'n_c': 1000, 'stack_size': 16005, 'sizeof int': 4, 'sizeof double': 8}


# Predictors and features

## Combine predictors and incorporate GPU properties 

In [4]:
%%html
<style>
table {margin-left: 0 !important;}
</style>


#### Input parameters "raw"

| kernel parameters | (raw kernel specifications) |
| :-----------------: | :---------------: |
| m, n, k           |       |
| algorithm         | 0:tiny, 1:small, 2:medium, 3:largeDB1, 4:largeDB2 |
| tile_m, tile_n    |       |
| w, v              |       |
| threads_per_blk   |       |
| grouping          | loosely corresponds to nrun |
| minblocks         |       |

| compilation       | (obtained from compilation log) |
| ----------------- |---------------|
| regs_per_thread   |               |
| nbytes_smem       |               |
| nbytes_cmem       |               |


| autotuning        |  |
| ----------------- |---------------|
| stack size        |       |


| GPU               | (obtained from CUDA) |
| ----------------- |---------------|

__________

#### Input parameters "derived"

| ?? | depends on |
| :----------------- | :--------------- |
| size_a, size_b, size_c       | m, n, k|
| need_sync                    |        |  
| ru_param_stack_unroll_factor |        |

| Launch configuration | depends on |
| ----------------- |---------------|
| nblks             | stack_size, grouping |
| warps_per_blk     |                      |
| nwarps    |                      |
| nthreads    |                      |
| sm_desired    |                      |
| smem_per_block    |                      |
| nblocks_per_sm_lim_blks_warps    |                      |
| nblocks_per_sm_lim_reg    |                      |
| nblocks_per_sm_lim_smem    |                      |
| nblocks_per_sm    |                      |
| nwarps_per_sm    |                      |
| occupancy    |                      |
       
| resource usage estimation (tiny) | depends on |
| ----------------- |---------------|
| ru_tiny_max_parallel_work | |
| ru_tiny_min_threads |
| ru_tiny_max_threads |
| ru_tiny_buf_size         |       |
| ru_tiny_smem_per_block             |  |
| ru_tiny_max_parallel_work         |       |
| ru_tiny_min_threads         |       |
| ru_tiny_max_threads         |       |
| ru_tiny_buf_size         |       |
| ru_tiny_smem_per_block         |       |

| resource usage estimation (small, medium) | depends on |
| ----------------- |---------------|
| ru_smallmed_max_parallel_work | |
| ru_smallmed_tm_max | |
| ru_smallmed_tn_max | |
| ru_smallmed_unroll_factor_c | |
| ru_smallmed_cmax | |
| ru_smallmed_rmax | |
| ru_smallmed_loop_matmul | |
| ru_smallmed_max_parallel_work | |
       
| resource usage estimation (tiny, small, medium) | depends on |
| ----------------- |---------------|
| ru_tinysmallmed_unroll_factor_a |        |
| ru_tinysmallmed_unroll_factor_b |        |


| resource usage estimation (largeDB1, largeDB2) | depends on |
| ----------------- |---------------|
| ru_large_Pa | m * w |
| ru_large_Pb | w * n |
| ru_large_Pc  | | 
| number of P_a, P_b | | 
| ru_large_unroll_factor_a | | 
| ru_large_unroll_factor_b | | 
| ru_large_unroll_factor_c | | 
| ru_large_loop_matmul | | 
       
| resource usage estimation (small, medium, largeDB1, largeDB2) | depends on |
| ----------------- |---------------|
| ru_smallmedlarge_T|        |

__________

#### Outcome-variable

1. perf (Gflop/s)
2. performance squared [Gflop/s]^2
3. boxcox(perf) [?]


![parameters dependency graph](ml_params.png)

In [4]:
import math

def ceiling(x, step):
    return np.where(x % step == 0, x, x + step - x % step)

def flooring(x, step):
    return np.where(x % step == 0, x, x - x % step)

def ceil_division(a, b):
    return (a + b - 1) // b

def add_matrix_sizes(df):
    df['size_a'] = df['m'] * df['k']
    df['size_b'] = df['k'] * df['n']
    df['size_c'] = df['m'] * df['n']

def add_launch_pars(df): 
    # Need sync? (relevant for tiny, small, medium) 
    # (mn > warp_size || mk > warp_size || kn > warp_size || threads > warp_size);
    df['need_sync'] = (np.where(df['size_c'] > gpu['Threads per Warp'], True, False)
                       | np.where(df['size_a'] > gpu['Threads per Warp'], True, False) 
                       | np.where(df['size_b'] > gpu['Threads per Warp'], True, False)
                       | np.where(df['threads_per_blk'] > gpu['Threads per Warp'], True, False)).tolist()
    
    # Launching parameters
    df['nblks'] = ceil_division(autotuning['stack_size'], df['grouping'])
    df['warps_per_blk'] = np.floor(df['threads_per_blk'] / gpu['Threads per Warp'])
    df['nwarps'] = df['warps_per_blk'] * df['nblks']
    df['nthreads'] = df['threads_per_blk'] * df['nblks']

def add_occupancy_estimation(df, one_col): 
    df['sm_desired'] = ceil_division(df['nblks'], df['minblocks'])
    
    # Resource occupations (warps, blocks), (Follows CUDA calculator sheet)
    df['nblocks_per_sm_lim_blks_warps'] = np.minimum(one_col * gpu['Max Thread Blocks per Multiprocessor'], \
                                                     np.floor(gpu['Max Warps per Multiprocessor'] / df['warps_per_blk']))

    # Resource occupations (registers)
    if 'regs_per_thread' in df.columns:
        df['i1'] = flooring(gpu['Max Registers per Thread Block'] * one_col / ceiling(df['regs_per_thread'] * gpu['Threads per Warp'], gpu['Register allocation unit size'] * one_col), 
                            gpu['Warp allocation granularity'] * one_col)
        df['nblocks_per_sm_lim_reg'] = np.floor(df['i1'] / df['warps_per_blk']) * \
                                       math.floor(gpu['Registers per Multiprocessor'] / gpu['Max Registers per Thread Block'])
        df = df.drop('i1', axis=1)
        
    # Resource occupations (shared memory)
    if 'nbytes_smem' in df.columns:
        df['smem_per_block'] = ceiling(df['nbytes_smem'], gpu['Shared Memory allocation unit size'])
        df['nblocks_per_sm_lim_smem'] = np.floor(one_col * gpu['Shared Memory per Multiprocessor (bytes)'] / df['smem_per_block'], one_col)
        
    # Aggregate 
    if 'nblocks_per_sm_lim_blks_warps' in df.columns and 'nblocks_per_sm_lim_reg' in df.columns and 'nblocks_per_sm_lim_smem' in df.columns :
        df['nblks_per_sm'] = df[['nblocks_per_sm_lim_blks_warps', 'nblocks_per_sm_lim_reg', 'nblocks_per_sm_lim_smem']].min(axis=1)
        df['nwarps_per_sm'] = df['nblks_per_sm'] * df['warps_per_blk']
        df['nsm'] = ceiling(df['nblks'], df['nblks_per_sm'])
        df['ngpu'] = ceiling(df['nsm'], gpu['Multiprocessors'])
        df['occupancy'] = df['nwarps_per_sm'] / gpu['Max Warps per Multiprocessor']
        
def add_ru_common(df): 
    # Loop counts
    df['ru_param_stack_unroll_factor'] = df['grouping'] // df['threads_per_blk']

def add_ru_tiny(df):
    # Resource usage estimation and loop counts (tiny) 
    df['ru_tinysmallmed_unroll_factor_a'] = df['size_a'] // df['threads_per_blk']
    df['ru_tinysmallmed_unroll_factor_b'] = df['size_b'] // df['threads_per_blk']
    df['ru_tiny_max_parallel_work'] = df[['grouping', 'size_a', 'size_b', 'size_c']].max(axis=1)
    df['ru_tiny_min_threads'] = df['size_b']
    df['ru_tiny_max_threads'] = ceiling(df['ru_tiny_max_parallel_work'], gpu['Threads per Warp'])
    df['ru_tiny_buf_size'] = df['k'] * (df['m'] + df['n'])
    if 'grouping' in df.columns:
        df['ru_tiny_smem_per_block'] = (df['ru_tiny_buf_size'] * autotuning['sizeof double']) + (npars * df['grouping'] * autotuning['sizeof int']) 
    
def add_ru_smallmed(df):
    # Resource usage estimation and loop counts (small, medium) 
    df['ru_smallmed_tm_max'] = df['m']
    df['ru_smallmed_tn_max'] = df['n']
    df['ru_smallmed_unroll_factor_c'] = df['size_c'] // df['threads_per_blk']
        
    if 'tile_m' in df.columns and 'tile_n' in df.columns:
        df['ru_smallmed_cmax'] = ceil_division(df['n'], df['tile_n'])
        df['ru_smallmed_rmax'] = ceil_division(df['m'], df['tile_m'])
        df['ru_smallmedlarge_T'] = df['tile_m'] * df['tile_n']
        df['ru_smallmed_loop_matmul'] = df['k'] * df['tile_m'] * df['tile_n']
            
        df['intermediate'] = df['ru_smallmed_cmax'] * df['ru_smallmed_rmax']
        df['ru_smallmed_max_parallel_work'] = df[['grouping', 'size_a', 'size_b', 'size_c', 'intermediate']].max(axis=1)
        df = df.drop('intermediate', axis=1)
        
        df['ru_smallmed_min_threads'] = df['ru_smallmed_cmax'] * df['ru_smallmed_rmax']
        df['ru_smallmed_max_threads'] = ceiling(df['ru_smallmed_max_parallel_work'], gpu['Threads per Warp'])
        
        df['intermediate1'] = df['size_a'] + df['k'] * df['tile_n'] * df['ru_smallmed_cmax']
        df['intermediate2'] = df['tile_m'] * df['ru_smallmed_rmax'] * df['k'] + 1
        df['ru_smallmed_buf_size'] = df[['size_c', 'intermediate1', 'intermediate2']].max(axis=1)
        df = df.drop('intermediate1', axis=1)
        df = df.drop('intermediate2', axis=1)

        df['ru_smallmed_smem_per_block'] = (df['ru_smallmed_buf_size'] * autotuning['sizeof double']) + (npars * df['grouping'] * autotuning['sizeof int'])
        
def add_ru_large(df):
    # Resource usage estimation and loop counts (largeDB1, largeDB2) 
    df['ru_large_Pa'] = df['m'] * df['w']  # Input slab sizes 
    df['ru_large_Pb'] = df['w'] * df['n']
    df['ru_large_Pc'] = df['m'] * df['v']  # Output slabs
    df['ru_large_unroll_factor_a'] = df['ru_large_Pa'] // df['threads_per_blk']
    df['ru_large_unroll_factor_b'] = df['ru_large_Pb'] // df['threads_per_blk']
    df['ru_large_unroll_factor_c'] = df['ru_large_Pc'] // df['threads_per_blk']
    df['ru_large_loop_matmul'] = df['w'] * df['tile_m'] * df['tile_n']

def add_derived_parameter(df):
    one_col = np.ones(len(df['algo']))
    add_matrix_sizes(df)
    add_launch_pars(df)
    add_occupancy_estimation(df, one_col)
    add_ru_common(df)
    add_ru_tiny(df)
    add_ru_smallmed(df)
    add_ru_large(df)

# Call
add_derived_parameter(pars_autotuned)
add_derived_parameter(pars_autotuning)

## Data Transformations

look at: sklearn.preprocessing

#### Feature scaling 
from sklearn.preprocessing import scale
Xs = scale(X)

#### Mean normalization

#### Log-scale or other transf. 

Box-Cox, power transform in order to better capture large peformances

In [ ]:
# Scaling

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)

### Groups of predictors (for conveniance) 

In [ ]:
# Raw predictors
mnk = ['m', 'n', 'k']
kernel_pars = ['algo', 'tile_m', 'tile_n', 'v', 'w', 'threads_per_blk', 'minblocks', 'grouping'] 
memory = ['nbytes_smem', 'regs_per_thread']
predictors_raw = kernel_pars + memory

# Auxiliaries 
sync = ['need_sync']
ru = ['i1', ]
ru_redundant = ['nthreads', 'occupancy', 'nblocks_per_sm', 'nblks']    # with: nwarps, nwarps_per_sm, nwarps_per_sm, nwarps
ru_tiny = ['ru_tiny_max_parallel_work', 'ru_tiny_min_threads', 'ru_tiny_max_threads', 'ru_tiny_buf_size', 'ru_tiny_smem_per_block']
ru_smallmed = ['ru_smallmed_tm_max', 'ru_smallmed_tn_max', 'ru_smallmed_cmax', 'ru_smallmed_rmax', 'ru_smallmed_max_parallel_work']
nblks_lims = ['nblocks_per_sm_lim_blks_warps', 'nblocks_per_sm_lim_reg', 'nblocks_per_sm_lim_smem']
cmem = ['nbytes_cmem']
sm_desired = ['sm_desired']

# Features 
matrix_sizes = ['size_a', 'size_b', 'size_c']
kernel_pars_red = ['algo', 'tile_m', 'tile_n', 'v', 'w', 'minblocks', 'grouping']
launch_pars = ['warps_per_blk', 'nwarps']
occupancy = ['smem_per_block', 'nwarps_per_sm']
predictor_features = kernel_pars_red + launch_pars + occupancy

# Outcome
perf = ['perf (Gflop/s)']

# Data visualization / Exploratory analysis

### Description

In [5]:
print('--- Pars autotuned:')
print('Numer of column:', len(pars_autotuned.columns), '\nNumber of rows:', len(pars_autotuned['algo']))
print('Column names:\n', pars_autotuned.columns)
print('\n--- Pars autotuning:')
print('Numer of column:', len(pars_autotuning.columns), '\nNumber of rows:', len(pars_autotuning['algo']))
print('Column names:\n', pars_autotuning.columns)

--- Pars autotuned:
Numer of column: 46 
Number of rows: 1509
Column names:
 Index(['algo', 'm', 'n', 'k', 'tile_m', 'tile_n', 'w', 'v', 'threads_per_blk',
       'grouping', 'minblocks', 'perf (Gflop/s)', 'size_a', 'size_b', 'size_c',
       'need_sync', 'nblks', 'warps_per_blk', 'nwarps', 'nthreads',
       'sm_desired', 'nblocks_per_sm_lim_blks_warps',
       'ru_param_stack_unroll_factor', 'ru_tinysmallmed_unroll_factor_a',
       'ru_tinysmallmed_unroll_factor_b', 'ru_tiny_max_parallel_work',
       'ru_tiny_min_threads', 'ru_tiny_max_threads', 'ru_tiny_buf_size',
       'ru_tiny_smem_per_block', 'ru_smallmed_tm_max', 'ru_smallmed_tn_max',
       'ru_smallmed_unroll_factor_c', 'ru_smallmed_cmax', 'ru_smallmed_rmax',
       'ru_smallmedlarge_T', 'ru_smallmed_loop_matmul', 'intermediate',
       'ru_smallmed_max_parallel_work', 'ru_large_Pa', 'ru_large_Pb',
       'ru_large_Pc', 'ru_large_unroll_factor_a', 'ru_large_unroll_factor_b',
       'ru_large_unroll_factor_c', 'ru_large_loop

In [ ]:
# Describe pars_autotuned
for i, pa in enumerate(pars_autotuning): 
    print("\n\nParameters autotuning (", i, ")")
    display(pa[mnk + matrix_sizes + perf].describe())
    display(pa[kernel_pars].describe())
    display(pa[memory].describe())
    display(pa[launch_pars].describe())
    display(pa[ru_tiny].describe())
    display(pa[ru_smallmed].describe())
    display(pa[occupancy].describe())

In [ ]:
# Describe pars_autotuned
display(pars_autotuned[mnk + matrix_sizes + perf].describe())
display(pars_autotuned[kernel_pars].describe())
display(pars_autotuned[launch_pars].describe())
display(pars_autotuned[ru_tiny].describe())
display(pars_autotuned[ru_smallmed].describe())

### Data Profiling

#### Observations 
* There are A LOT of unnecessary (redundant) predictor variables, do not forget to sithe through 
* nbloks_per_sm is strongly positively correlated with performance (duh) 

In [ ]:
import pandas_profiling 
pandas_profiling.ProfileReport(pars_autotuned)

In [ ]:
pandas_profiling.ProfileReport(pars_autotuning[3])

In [ ]:
pandas_profiling.ProfileReport(pars_autotuning[3][predictors_raw + perf])

In [ ]:
pandas_profiling.ProfileReport(pars_autotuning[3][predictor_features + perf])

### Histograms

In [ ]:
# Histograms with Bokeh
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook, show
output_notebook()

num_bins = 200 
toplot = pars_autotuning[3]['perf (Gflop/s)']

# Create histogram
hist, edges = np.histogram(toplot, bins=num_bins)
df_hist = pd.DataFrame({'hist': hist, 'left': edges[:-1], 'right': edges[1:]})
source = ColumnDataSource(df_hist)

# Create tool 
hover = HoverTool(tooltips=[('# occurences', '@hist'), ('low', '@left'), ('high', '@right')])

# Create the figure
p = figure(plot_width=800, plot_height=800, title="Performance histogram",
           toolbar_location=None, tools="")
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "perf (GFlop/s)"
p.xaxis.major_label_orientation = 1.2
p.yaxis.axis_label = "# occurrences"
p.quad(source=source, bottom=0, top='hist', left='left', right='right', fill_color='blue')
p.add_tools(hover)

show(p)

### Top performances

#### Observations 
* well at least there are still quite a few rows even in the top 0.5%
* hmm ... I don't really see any "consensus" in the top performances for larger kernels. Even in the top 0.5%, there are multiple kernels and multiple "main parameters" 
* it seems as though "grouping" and "minblocks" are the least influential raw parameters

In [ ]:
# Top slices of perf. distribution
pars_autotuning_top = {
    2: list(), 
    1: list(), 
    0.5: list()
}
#pars_autotuning_top2 = list() # top 2%
#pars_autotuning_top1 = list() # top 1%
#pars_autotuning_top05 = list() # top 0.5%
for i, pa in enumerate(pars_autotuning):
    print('\n\n---------', i)
    max_perf = float(pa[perf].max())
    max_perf_idx = pa[perf].idxmax()
    max_perf_row = pa.loc[max_perf_idx]
    max_perf_cond = max_perf_row[mnk + kernel_pars + perf]
    display(max_perf_cond)
    for perc in pars_autotuning_top.keys():
        lim = max_perf - max_perf*perc/100
        blob = pa.loc[pa['perf (Gflop/s)'] >= lim]
        print('\ntop', perc, '%')
        display(blob[predictors_raw + perf].describe())
        pars_autotuning_top[perc].append(blob)

### Pair plot 

#### Raw predictors & performance

* perf spreads and diminishes with tile_m, tile_n 
* aside from that, basically there's nothing to see ... :(  

#### Features and performance 

* well no, it does not look a lot better ... 

In [ ]:
import seaborn as sns
pars_0 = pars_autotuning[3].replace(to_replace={np.NaN: 0})
sns.pairplot(pars_0[predictors_raw + perf])

In [ ]:
import seaborn as sns
pars_0 = pars_autotuning[3].replace(to_replace={np.NaN: 0})
sns.pairplot(pars_0[predictor_features + perf])

### Box and scatter plots

Perhaps plots make more sense once some values are binned? 

In [ ]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
output_notebook()

for f in predictor_features:
    print(f)
    p = figure(title=f)
    p.circle(x=pars_autotuning[3][f], y=pars_autotuning[3]['perf (Gflop/s)'], size=0.5)
    show(p)

## Rudimentary feature selection

* I'm guessing that univariate selection won't bring much as there are basically no real 1st order relationships between predictors and performance (cf pair plot) 

* Recursive feature elimination seems t make a lot more sense... but it has to be used with a model si j'ai bien compris ... 

In [ ]:
# Import `RandomForestClassifier`
from sklearn.ensemble import RandomForestClassifier

# Isolate Data, class labels and column values
X = iris.iloc[:,0:4]
Y = iris.iloc[:,-1]
names = iris.columns.values

# Build the model
rfc = RandomForestClassifier()

# Fit the model
rfc.fit(X, Y)

# Print the results
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rfc.feature_importances_), names), reverse=True))

In [ ]:
# Feature Importance with Extra Trees Classifier
from pandas import read_csv
from sklearn.ensemble import ExtraTreesClassifier
# load data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
# feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)

In [97]:
# To do with a model :
from sklearn.feature_selection import RFE
model = LogisticRegression()
rfe = RFE(model, 3) # 2nd arg = num features to retain
fit = rfe.fit(X, Y)
print("Num Features: %d") % fit.n_features_
print("Selected Features: %s") % fit.support_
print("Feature Ranking: %s") % fit.ranking_

# Create the RFE object and rank each pixel
svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(X, y)
ranking = rfe.ranking_.reshape(digits.images[0].shape)

NameError: name 'LogisticRegression' is not defined

In [ ]:
# RFE with CV
# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
# Feature Extraction with PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
fit = pca.fit(X)
# summarize components
print("Explained Variance: %s") % fit.explained_variance_ratio_
print(fit.components_)

# Predictive Modeling 

Modeling for multiple m,n,ks at a time: 
    Add mnk OR sizes (which?)
    Considering adding need_sync

## Variables

In [ ]:
# Predictor 
X = pars_autotuning[3][predictor_features]
n_features = len(list(X.columns))
print('Predictor variables:\n', X.columns)

# Outcome
Y = pars_autotuning[3][perf]
maxperf = float(Y.max(axis=0))
print('\nOutcome variable:\n', Y.columns, "with max. value =", maxperf)

## Utilities

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split  
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook, show
output_notebook()

def print_error(y_true, y_pred):
    print('Mean Absolute Error:     {:>7.4f}'.format(metrics.mean_absolute_error(y_true, y_pred)))
    print('Mean Squared Error:      {:>7.4f}'.format(metrics.mean_squared_error(y_true, y_pred)))
    print('Root Mean Squared Error: {:>7.4f}\n'.format(np.sqrt(metrics.mean_squared_error(y_true, y_pred))))

def display_tree(model, model_name):
    dot_data = tree.export_graphviz(model, out_file=None, filled=True, 
                                    leaves_parallel=True, feature_names=list(X.columns))
    graph = graphviz.Source(dot_data)
    graph.render(model_name)
    return graph

def perf_pred_hist(y):
    # Create histogram and hover tool 
    num_bins = 200
    hist, edges = np.histogram(y, bins=num_bins)
    df_hist = pd.DataFrame({'hist': hist, 'left': edges[:-1], 'right': edges[1:]})
    source = ColumnDataSource(df_hist)
    hover = HoverTool(tooltips=[('# occurences', '@hist'), ('low', '@left'), ('high', '@right')])

    # Create the figure
    p = figure(plot_width=800, plot_height=800, title="Predicted performance histogram",
               toolbar_location=None, tools="")
    p.xgrid.grid_line_color = None
    p.xaxis.axis_label = "predicted perf (GFlop/s)"
    p.xaxis.major_label_orientation = 1.2
    p.yaxis.axis_label = "# occurrences"
    p.quad(source=source, bottom=0, top='hist', left='left', right='right', fill_color='blue')
    p.add_tools(hover)
    return p

def perf_pred_scatter(y, title, maxperf, maxperf_pred): 
    p = figure(plot_width=800, plot_height=800, title=title)
    max_x = len(y)
    x = np.arange(max_x)
    p.circle(x=x, y=y, size=4)
    p.line(x=[0, max_x], y=[maxperf, maxperf], legend="max. perf.", color='red')
    p.line(x=[0, max_x], y=[maxperf_pred, maxperf_pred], legend="max. leaf label", color='green')
    p.legend.location = "bottom_right"
    return p 

def scatter_top(y, title):
    top_m = 50
    top_m_idx = np.argpartition(-y, top_m)[:top_m]
    y_predmax = Y.iloc[top_m_idx]
    p = perf_pred_scatter(y_predmax.values.ravel(), title, maxperf, np.NaN)
    show(p)

def fit_model(model, x, y): 
    
    # X, Y, split, model
    X_train, X_test, y_train, y_test = train_test_split(x, y.values.ravel(), test_size=0.20, random_state=0)  

    # Fit
    model.fit(X_train, y_train)
    print('\n-------------------')
    print(model, '\n')
    print('Feature importance:')
    print(model.feature_importances_, '\n')

    # Training error
    y_train_pred = model.predict(X_train)
    print('Training error:')
    print_error(y_train, y_train_pred)

    # Test
    y_pred = model.predict(X_test)  
    print('Testing error:')
    print_error(y_test, y_pred)
    
    return np.concatenate((y_train_pred, y_pred), axis=0)


## Decision Tree & Co 

#### Observations 

* DT with friedman mse is basically the same as regular mse, I stopped looking at it 
* top-classes are too large (size 3'200 and 800 for mse, mae respectively) 
    * starting from dpeth = 8-9, this gets more reasonable
* Maybe DTs are just not discriminatory enough??  hmm ... 
* IDEAS: get a 2 level tree? (a) identify top class, (b) discriminate further inside of top-class ? 

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
import graphviz

# Hyperparameters
tree_depth = 10
min_samples_split = 20 
min_samples_leaf = 1
top_k = 20

def tree_model(model, model_name, X, Y): 
    
    perf = fit_model(model, X, Y)
    graph = display_tree(model, model_name)
    p = perf_pred_hist(perf)
    show(p)

    print('highest', top_k, ':')
    top = -np.partition(-perf, top_k)[:top_k]
    print(top)

    top_m = 542
    top_m_idx = np.argpartition(-perf, top_m)[:top_m]
    y_predmax = Y.iloc[top_m_idx]
    
    p = perf_pred_scatter(y_predmax.values.ravel(), model_name, maxperf, top[0])
    show(p)
    return graph

# Default DT 
model_DT = DecisionTreeRegressor(criterion='mse', max_depth=tree_depth)
graph_DT = tree_model(model_DT, "model_DT", X, Y)

# DT with MAE criterion 
#model_DTa = DecisionTreeRegressor(criterion='mae', max_depth=tree_depth) 
#graph_DTa = tree_model(model_DTa, "model_DTa", X, Y)

# DT with Friedman criterion 
#model_DTf = DecisionTreeRegressor(criterion='friedman_mse', splitter='best', max_depth=tree_depth)
#graph_DTf = tree_model(model_DTf, "model_DTf", X, Y)

In [ ]:
graph_DT

In [ ]:
graph_DTa

### Random Forest (ensemble method) 

#### Observations
the feature importances are completely off: mostly tile_m and tile_n are used :( 
I need to write a custom accuracy: 
    are the k-top among the p% of max perf? och ... 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_RF = RandomForestRegressor(bootstrap=False, n_estimators=100, criterion='mse', 
                                 max_features='auto', max_depth=None, 
                               min_samples_split=20, min_samples_leaf=1)

# Fit
perf_RF = fit_model(model_RF, X, Y)

# hist 
p = perf_pred_hist(perf_RF)
show(p)

In [ ]:
# scatter top 
top_m = 50
top_m_idx = np.argpartition(-perf_RF, top_m)[:top_m]
y_predmax = Y.iloc[top_m_idx]

p = perf_pred_scatter(y_predmax.values.ravel(), "RF", maxperf, np.NaN)
show(p)

### Decision tree with AdaBoost (ensemble method) 

#### Observations


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

model_AB = AdaBoostRegressor(DecisionTreeRegressor(),
                             n_estimators=300, random_state=0)
perf_AB = fit_model(model_AB, X, Y)
p = perf_pred_hist(perf_AB)
show(p)

In [ ]:
# scatter top 
top_m = 50
top_m_idx = np.argpartition(-perf_AB, top_m)[:top_m]
y_predmax = Y.iloc[top_m_idx]

p = perf_pred_scatter(y_predmax.values.ravel(), "AB", maxperf, np.NaN)
show(p)

### Gradient Boosting regressor (ensemble method) 

#### Observations

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model_GBR = GradientBoostingRegressor(n_estimators=500, random_state=0, learning_rate=0.1)
perf_GBR = fit_model(model_GBR, X, Y)
p = perf_pred_hist(perf_GBR)
show(p)

In [ ]:
# scatter top 
top_m = 50
top_m_idx = np.argpartition(-perf_GBR, top_m)[:top_m]
y_predmax = Y.iloc[top_m_idx]

p = perf_pred_scatter(y_predmax.values.ravel(), "GBR", maxperf, np.NaN)
show(p)

### Bagging regressor (over DT) 

#### Observations 

In [ ]:
from sklearn.ensemble import BaggingRegressor

model_BR = BaggingRegressor()
perf_BR = fit_model(model_BR, X, Y)
show(perf_pred_hist(perf_BR))
show(scatter_top(perf_BR))

### Polynomial regression

#### Observations

### SVR (Support Vector Regression) 

#### Observations

In [ ]:
from sklearn.svm import SVR

#svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
#perf_svr_rbf = fit_model(svr_rbf, X, Y)
#p = perf_pred_hist(perf_svr_rbf)
#show(p)

svr_lin = SVR(kernel='linear', C=1e3)
perf_svr_lin = fit_model(svr_lin, X, Y)
p = perf_pred_hist(perf_svr_lin)
show(p)

# scatter top 
top_m = 50
top_m_idx = np.argpartition(-perf_svr_lin, top_m)[:top_m]
y_predmax = Y.iloc[top_m_idx]

p = perf_pred_scatter(y_predmax.values.ravel(), "SVR-lin", maxperf, np.NaN)
show(p)

#svr_poly = SVR(kernel='poly', C=1e3, degree=2)
#perf_svr_poly = fit_model(svr_poly, X, Y)
#p = perf_pred_hist(perf_svr_poly)
#show(p)

### Kernel Ridge Regression

### GPR (Gaussian Process Regression)

### MLP (Multi Layer Perceptron) 